# Mean minimal spanning tree length

### Set up workspace

In [18]:
pip install ipympl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.2 MB/s eta 0:00:00


Import supporting libraries

In [29]:
# imports
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from itertools import combinations_with_replacement as cwr
from functools import reduce
import sys
import json
import ipywidgets as widgets


Choose $k$, the number of particle types; i.e. the random graph will be $k$-partite.

In [20]:
k = 4;
assert k == 4, f"Interaction strength matrix is not defined for k = {k}"

### Creating helper functions

#### Factorial computation

In [21]:
def factorial(n: int):
  assert isinstance(n, int), f"Factorial function only accepts integer inputs.\n(Received {type(n)})";
  if n < 0: return np.nan;
  return 1 if ((n==1)|(n==0)) else n*factorial(n-1);

#### Define function $f(p,x)$ for each summand $\{x: \langle x | 1 \rangle = n \}$

In [22]:
def eff(x: np.ndarray, V: np.ndarray):
  xm = [float(num) for num in x - 1];
  y = np.matmul(V, x)**np.array(xm)

  # compute product of components of y
  eff = reduce(
      lambda a,b: a*b,
      y,
      1
    )

  return eff;

Create summand function

In [23]:
def summand(x: np.ndarray, V: np.ndarray):

  # compute product of powers of components of V*x
  f = eff(x,V);

  # compute leading constant numerator
  c0 = factorial(int(x.sum()));

  # compute denominator, x! = x_1! * x_2! * ... * x_k!
  c1 = reduce(
      lambda a,b: a*b,
      map(
        lambda en: factorial(int(en)),
        x
      ),
      1
    )

  return c0 * f / c1;

#### Compute partitions

In [24]:
def get_partitions(n, k):
  '''
  number of ways to designate your n particles into k urns:
  place k-1 separators among/around the ordered n particles (n+1 choices)
  the particles above the highest separator are urn 1 members,
  the particles between the first & second sep are in urn 2, ...
  the placement of separators is done by choosing one of the n+1 spaces k-1 times
  with replacement and without regard to order; use the itertools function for this.
  '''
  raw_parts = list(
      cwr(np.arange(0, n+1), k-1)
    ) # weak

    # first bin:  rp[0]
    # middle bins:  np.diff(rp)
    # final bin:  n - rp[-1]

  # would be nice to perform this process using map, not listing
  out = np.empty((len(raw_parts), k));
  for ent, y in enumerate(raw_parts):
    out[ent, :] = [y[0]] + np.diff(y).tolist() + [n-y[-1]]

  return out;

#### Code for generating LHS of (23)

In [ ]:
def compute_lhs(n, k, V):
  lhs = 0;
  for x in get_partitions(n,k):
    lhs += summand(x, V)
  return lhs;

#### Code for generating RHS of (23)

In [25]:
def compute_rhs(n,k):
  rhs = k * (k-1)**(n-1) * n**(n-4)
  return rhs;

## Plotting

### Surface plot

Now we get into actually computing the quantities of interest. Resolution manages the resolution of the surface plot below.

In [9]:
# resolution of the 2-d plots
resolution = 50

Begin by defining the parameters $n$ (total number of particles / nodes in the graph), $p$, $q$, and $r$ (the inter-group interaction strength parameters)
We define now the domain for the independent, nonconstant variables:

In [10]:
lower_bound = 0.5;
upper_bound = 20;

domain_x = np.linspace(lower_bound, upper_bound, resolution);
domain_y = np.linspace(lower_bound, upper_bound, resolution);

[X,Y] = np.meshgrid(domain_x, domain_y)
F = np.zeros(X.shape)

Enable and generate widgets

In [26]:
%matplotlib widget

p_slider = widgets.FloatSlider(
    value=1,
    min=lower_bound,
    max=upper_bound,
    step=0.1,
    description='p:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

q_slider = widgets.FloatSlider(
    value=1,
    min=lower_bound,
    max=upper_bound,
    step=0.1,
    description='q:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

r_slider = widgets.FloatSlider(
    value=1,
    min=lower_bound,
    max=upper_bound,
    step=0.1,
    description='r:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

n_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description='n value:',
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


Compute the interaction strength matrix, then execute the computation steps.

#### Notice that the output matrix $F$ is the ratio fo the left- and right-hand sides of (24):

In [30]:

def run_plot(n,var3):
    global F

    r = var3;
    for eye, p in enumerate(domain_x):
      for jay, q in enumerate(domain_y):
        V = np.array([
          [0, p, q, r],
          [p, 0, r, q],
          [q, r, 0, p],
          [r, q, p, 0]
        ])

        G = compute_lhs(n,k,V)
        H = compute_rhs(n,k)
        F[eye, jay] = G/H;

    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                           linewidth=0, antialiased=False)

    ax.set_title(f"k={k}, n={n}\nG(p, q, {r})/H")
    plt.show()

We plot here the results:

In [31]:
widgets.interact(run_plot, n=n_slider, var3=r_slider)

interactive(children=(IntSlider(value=5, continuous_update=False, description='n value:', max=20, min=1), Floa…

<function __main__.run_plot(n, var3)>

## Computing sums of $f(p, x)$ and $f(\bar{p}, x)$

The function `summand` above computes $f(p,x)$ where $V$ is given as a function of $p$, $q$, and $r$.

We can compute $f(\bar{p},x)$ as well by passing in $\mathbb{1}_k * \mathbb{1}_k^T - I_k$ in place of $V$ and prepending a factor of $\left(\frac{p+q+r}{3}\right)^{n-4}$:

### Adding helper functions for $f(\bar{p}, x)$

Compute $f(\bar{p}, x)$

In [32]:
def eff_bar(x: np.ndarray, p, q, r):
  C = np.mean([p, q, r]);
  f = eff(x, np.ones((k,k)) - np.eye(k));
  f_bar = C**(np.sum(x) - 4.0) * f;
  return f_bar;

def summand_bar(x, p, q, r):
  f_bar = eff_bar(x,p,q,r);

  # compute leading constant numerator
  c0 = factorial(int(x.sum()));

  # compute denominator, x! = x_1! * x_2! * ... * x_k!
  c1 = reduce(
      lambda a,b: a*b,
      map(
        lambda en: factorial(int(en)),
        x
      ),
      1
    )

  return c0 * f_bar / c1;

Define the plot

In [33]:
def eff_vs_eff_bar(var1, var2, var3):

    [p, q, r] = var1, var2, var3;
    V = np.array([
      [0, p, q, r],
      [p, 0, r, q],
      [q, r, 0, p],
      [r, q, p, 0]
    ])

    n_domain = np.arange(1, 10);
    series1 = np.zeros(n_domain.shape);
    series2 = np.zeros(n_domain.shape);
    for ent, n in enumerate(n_domain):
        summa = 0; # sum of f(p,x)
        sum_bar = 0; # sum of f(p-bar, x)
        for x in get_partitions(n,k):
            # compute denominator, x! = x_1! * x_2! * ... * x_k!
            c1 = reduce(
              lambda a,b: a*b,
              map(
                lambda en: factorial(int(en)),
                x
              ),
              1
            )

            summa += eff(x, V) / c1;
            sum_bar += eff_bar(x, p, q, r) / c1;

        # record results, normalizing by n!
        series1[ent] = summa;
        series2[ent] = sum_bar;

    # plot the two series
    fig, ax = plt.subplots()
    ax.scatter(n_domain, series1, color="tab:blue")
    ax.set_yscale('log')

    ax.set_title(f"k={k}\n(p,q,r) = ({p},{q},{r})")
    ax.set_xlabel("n")
    ax.set_ylabel("f(p,x)", color="blue")
    # ax.xaxis.label.set_color('red')

    ax.tick_params(axis='y')

    ax2 = ax.twinx();
    # ax2.scatter(n_domain, series2)
    ax2.scatter(n_domain, series2, color="tab:orange", marker="*")
    ax2.set_ylabel("f(p-bar, x)", color="orange")
    ax2.set_yscale('log')

    # comment this out to have independent y-axes
    ax2.sharey(ax)

    plt.show()

### The plot

In [34]:
widgets.interact(eff_vs_eff_bar, var1=p_slider, var2=q_slider, var3=r_slider)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='p:', max=20.0, min=0.5, rea…

<function __main__.eff_vs_eff_bar(var1, var2, var3)>